# Digging Beneath the Surface

*Benjamin Johnson*


*July 15, 2022*

[Home](https://fastdatascientist.github.io/Digging-Beneath-the-Surface/)

[About Me](https://fastdatascientist.github.io/Digging-Beneath-the-Surface/author.html)

[Previous Posts](https://fastdatascientist.github.io/Digging-Beneath-the-Surface/posts.html)

## Electric Vehicle (EV) vs Gas Break-Even Analysis

This year has been full of gloom-and-doom headlines regarding the economy. While I intend to dig into a history behind bear markets in a future article, today I'd like to focus on the spike in gas prices. The average price per gallon in Illinois had been hovering in the low-to-mid \$3's in 2021 and into early 2022, but it's jumped all the way up past $5. At the same time, auto manufacturers have been releasing an increased number of electric vehicle (EV) options. Although the purchase prices are steep, the operating and maintenance cost differential between gas and EV vehicles are higher than ever. Today, let's run a break-even analysis to see the long term cost benefits of purchasing an EV.

### Purchase Prices

In [6]:
# Purchase Prices for EV and Gas
ev_purch = 48490
gas_purch = 33950

Our first two variables are the purchase prices for our sample gas and electric powered vehicles. I selected a Mercedes-Benz A-Class and Tesla Model 3, both falling into the luxury sedan category.

### Estimated Usage

In [2]:
# Assume we drive this many miles per year
yearly_mi = 15000  

Next, we specify the number of miles we plan to drive each year. 

### Gas / Charging Cost Factors

In [3]:
# Gas / Electricity Cost Parameters
gas_per_gal = 5.15     # Cost per gallon at the gas station
gas_mpg     = 27       # 25 city / 35 highway miles per gallon
charge_cpm  = 0.041    # Estimated cost per mile for charging the EV

We'll use the quoted gas mileage for the Mercedes, current gas price, and approximate charging cost per mile for the Tesla to estimate our yearly energy costs to power the vehicles.

### Maintenance Factors

In [28]:
# Maintenance Factors
gas_maint = 0.0955       # Cost per mile for maintenance on gas vehicle
ev_maint  = 0.0777       # Cost per mile for maintenance on EV

Our next factors are related to maintenance. From some online research, estimates indicated that EVs can expected ~20% less maintenance due to fewer moving parts.

### Depreciation Factors

In [5]:
# Depreciation Factors
gas_depr = 0.13        # Yearly avg rate of depreciation for gas vehicle
ev_depr  = 0.15        # Yearly avg rate of depreciation for EV

We also need to account for depreciation if we were to sell the vehicles. While the last several years have actually seen used cars increase in value due to high demand & low supply, vehicles typically see significant depreciation over time. For simplicity, we're using average depreciation rates per year. The EV depreciation factor is set slightly higher due to initial government incentives and battery replacement over time.

### Rate of Return

In [91]:
# Rate of Return for difference in purchase price
ror = 0.10             # Assume we invest the difference in purchase price with a return of ror% annualized

Lastly, let's consider rate of return for the difference in purchase price of our EV vs gas vehicle. If invested, this will give us a return over time.

### Calculate Yearly Cash Flows from Years 0 - 15

Now that we've set up our cost factors, let's calculate our yearly expenses for both vehicles.

In [115]:
# Initialize numpy arrays for cash flows
import numpy as np
gas_arr = np.empty((0))
ev_arr = np.empty((0))

# Add purchase prices in year 0
gas_arr = np.append(gas_arr, gas_purch)
ev_arr = np.append(ev_arr, ev_purch)

# Calculate yearly cash flows
gas_cashflow = ((yearly_mi / gas_mpg) * gas_per_gal) + (gas_maint * yearly_mi) 
ev_cashflow = (yearly_mi * charge_cpm) + (ev_maint * yearly_mi) 

print("Gas vehicle costs $" + str("{:,.0f}".format(gas_cashflow)) + " per year.")
print("EV  vehicle costs $" + str("{:,.0f}".format(ev_cashflow)) + " per year.")
print("EV  savings are   $" + str("{:,.0f}".format(gas_cashflow - ev_cashflow)) + " per year.")
print()

# Append cash flows into lists for next 15 years
for i in range(15):
    gas_arr = np.append(gas_arr, gas_cashflow)
    ev_arr = np.append(ev_arr, ev_cashflow)

year_diff = ev_list - gas_list   
print(year_diff)

Gas vehicle costs $4,294 per year.
EV  vehicle costs $1,780 per year.
EV  savings are   $2,513 per year.

[14540.         -2513.11111111 -2513.11111111 -2513.11111111
 -2513.11111111 -2513.11111111 -2513.11111111 -2513.11111111
 -2513.11111111 -2513.11111111 -2513.11111111 -2513.11111111
 -2513.11111111 -2513.11111111 -2513.11111111 -2513.11111111]


It looks like choosing an EV will save us money over time, as it costs $2,513.11 less to maintain and operate per year. How long until we recoup our initial investment however? Let's take a look at the cumulative savings by year.

### Calculate Cumulative Savings every year

In [112]:
cum_diff = np.empty((0))
index = 0

for i in np.nditer(year_diff):
    if index == 0: val = i
    else: val = (val * (1+ror)) + i
    print("Year " + str(index) + " savings are " + str("${:,.0f}".format(val * -1)))
    index += 1
    cum_diff = np.append(cum_diff, val)

Year 0 savings are $-14,540
Year 1 savings are $-13,481
Year 2 savings are $-12,316
Year 3 savings are $-11,034
Year 4 savings are $-9,625
Year 5 savings are $-8,074
Year 6 savings are $-6,368
Year 7 savings are $-4,492
Year 8 savings are $-2,428
Year 9 savings are $-158
Year 10 savings are $2,340
Year 11 savings are $5,087
Year 12 savings are $8,108
Year 13 savings are $11,432
Year 14 savings are $15,089
Year 15 savings are $19,111


It takes just shy of 10 years for us to break even, meaning an EV might be a good investment if we were to drive it for at least that long. This does not consider what would happen if we were to sell the vehicle, so let's take a closer look at the difference with the sale price considered.

### Account for Sale Price after Depreciation

In [120]:
# Initialize numpy arrays for vehicle sales price by year
gas_sp = np.empty((0))
ev_sp = np.empty((0))

# Apply depreciation rate every year
for i in range(16):
    gas_new_price = gas_purch * (1-gas_depr) ** i
    ev_new_price = ev_purch * (1-ev_depr) ** i
    gas_sp = np.append(gas_sp, gas_new_price)
    ev_sp = np.append(ev_sp, ev_new_price)
    
print("Sales Values of Gas Vehicle: ")    
print(gas_sp)
print()
print("Sales Values of EV Vehicle: ")    
print(ev_sp)

Sales Values of Gas Vehicle: 
[33950.         29536.5        25696.755      22356.17685
 19449.8738595  16921.39025777 14721.60952426 12807.8002861
 11142.78624891  9694.22403655  8433.9749118   7337.55817327
  6383.67561074  5553.79778134  4831.80406977  4203.6695407 ]

Sales Values of EV Vehicle: 
[48490.         41216.5        35034.025      29778.92125
 25312.0830625  21515.27060312 18287.98001266 15544.78301076
 13213.06555914 11231.10572527  9546.43986648  8114.47388651
  6897.30280353  5862.707383    4983.30127555  4235.80608422]


Here we calculate the sales values for the vehicles by year.

In [125]:
sp_diff = ev_sp - gas_sp
cum_diff_revised = cum_diff - sp_diff

index = 0
for i in np.nditer(cum_diff_revised):
    print("Year " + str(index) + " revised savings are " + str("${:,.0f}".format(i * -1)))
    index += 1

Year 0 revised savings are $-0
Year 1 revised savings are $-1,801
Year 2 revised savings are $-2,979
Year 3 revised savings are $-3,612
Year 4 revised savings are $-3,762
Year 5 revised savings are $-3,480
Year 6 revised savings are $-2,802
Year 7 revised savings are $-1,755
Year 8 revised savings are $-358
Year 9 revised savings are $1,379
Year 10 revised savings are $3,452
Year 11 revised savings are $5,863
Year 12 revised savings are $8,622
Year 13 revised savings are $11,741
Year 14 revised savings are $15,240
Year 15 revised savings are $19,143


Now that we've considered the sale price, our break even point moves up slightly. It takes nearly 9 years for the EV savings to outweigh the higher initial cost.

### Conclusion

If you've made it this far, thanks for sticking around to the end! This article has been full of assumptions and limited options, so I'm sure some of you might be kicking and screaming. For instance, are luxury sedans really are only options? What does our break even point look like if we were to compare a Nissan Leaf or Chevy Bolt EV compared to a Toyota Camry or Ford Fusion? And how accurate (and consistent) were our cost factors anyway?
&nbsp;

All those questions aside, this was still a fun problem to analyze. As the number of EV options increase in the near future, I'll certainly revisit this analysis with updated considerations. For now, let's cross our fingers for gas prices to drop back down!
&nbsp;

Thanks again for reading, and tune in for the next post in late August!
&nbsp;

Until next time,

&nbsp;
Ben